In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
train = pd.read_excel("https://github.com/lulusAk/Flight_Tickets_Price_Predictions/raw/main/Data_Train.xlsx")
train.head()

In [ ]:
test = pd.read_excel("https://github.com/lulusAk/Flight_Tickets_Price_Predictions/raw/main/Test_set.xlsx")
test.head()

In [15]:
# Deleted column Date_of_Journey from train
train.drop(['Date_of_Journey'], axis=1, inplace=True)

# Deleted column Duration from train
train.drop(['Duration'], axis=1, inplace=True)

# Deleted column Arrival_Time from train
train.drop(['Arrival_Time'], axis=1, inplace=True)

# Changed Price from int64 to float
train['Price'] = train['Price'].astype('float')
train.head()


,Airline,Source,Destination,Route,Dep_Time,Total_Stops,Additional_Info,Price
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,non-stop,No info,3897.0
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,2 stops,No info,7662.0
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,2 stops,No info,13882.0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,1 stop,No info,6218.0
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,1 stop,No info,13302.0


In [16]:
train[['Stops','add']] = train['Total_Stops'].str.split(" ",expand=True)
train = train.drop(columns= ['add'])
train['Stops'] = train['Stops'].str.replace("non-stop","0")
train = train.drop(columns= ['Total_Stops'])
train.head()

,Airline,Source,Destination,Route,Dep_Time,Additional_Info,Price,Stops
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,No info,3897.0,0
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,No info,7662.0,2
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,No info,13882.0,2
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,No info,6218.0,1
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,No info,13302.0,1


In [17]:
# Changed Date_of_Journey from object to datetime
test['Date_of_Journey'] = pd.to_datetime(test['Date_of_Journey'], format='%m/%d/%Y', errors='coerce')

# Deleted column Date_of_Journey from test
test.drop(['Date_of_Journey'], axis=1, inplace=True)

# Deleted column Duration from test
test.drop(['Duration'], axis=1, inplace=True)

# Deleted column Arrival_Time from test
test.drop(['Arrival_Time'], axis=1, inplace=True)
test.head()

,Airline,Source,Destination,Route,Dep_Time,Total_Stops,Additional_Info
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,17:30,1 stop,No info
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,06:20,1 stop,No info
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,19:15,1 stop,In-flight meal not included
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,08:00,1 stop,No info
4,Air Asia,Banglore,Delhi,BLR → DEL,23:55,non-stop,No info


In [18]:
test[['Stops','add']] = test['Total_Stops'].str.split(" ",expand=True)
test = test.drop(columns= ['add'])
test['Stops'] = test['Stops'].str.replace("non-stop","0")
test = test.drop(columns= ['Total_Stops'])
test.head()

,Airline,Source,Destination,Route,Dep_Time,Additional_Info,Stops
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,17:30,No info,1
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,06:20,No info,1
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,19:15,In-flight meal not included,1
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,08:00,No info,1
4,Air Asia,Banglore,Delhi,BLR → DEL,23:55,No info,0


## Train Model

In [19]:
from pycaret.regression import *
s = setup(train, target = 'Price')

,Description,Value
0,session_id,8708
1,Target,Price
2,Original Data,"(10683, 8)"
3,Missing Values,True
4,Numeric Features,0
5,Categorical Features,6
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(7478, 174)"


In [20]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,1338.6099,5360135.0508,2301.8561,0.7464,0.2162,0.1500,1.0230
et,Extra Trees Regressor,1349.0625,5433963.2926,2319.2657,0.7420,0.2181,0.1507,1.1930
dt,Decision Tree Regressor,1348.8072,5438026.1635,2320.3166,0.7417,0.2183,0.1507,0.0390
gbr,Gradient Boosting Regressor,1542.2378,5717356.7083,2381.4862,0.7286,0.2351,0.1821,0.5560
br,Bayesian Ridge,1536.9156,5811006.0784,2402.4124,0.7236,0.2385,0.1803,0.1410
ridge,Ridge Regression,1537.5733,5828385.2000,2405.3890,0.7231,0.2382,0.1804,0.0260
lasso,Lasso Regression,1546.8177,5824278.9500,2405.1139,0.7230,0.2389,0.1816,0.3400
lightgbm,Light Gradient Boosting Machine,1416.0019,5988463.0643,2423.8409,0.7205,0.2205,0.1610,0.0740
llar,Lasso Least Angle Regression,1637.2499,6217055.4460,2484.3844,0.7046,0.2444,0.1937,0.0350
knn,K Neighbors Regressor,1453.0616,6565139.9000,2542.8765,0.6917,0.2322,0.1609,0.3930


In [21]:
print(best)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=8708, verbose=0, warm_start=False)


In [33]:
# tune model
tuned_best = tune_model(best)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1554.1970,5171729.7473,2274.1437,0.6982,0.2424,0.1897
1,1703.7789,7922834.0297,2814.7529,0.6584,0.2419,0.1895
2,1630.4075,5718259.8846,2391.2883,0.7049,0.2506,0.1982
3,1630.5996,5838828.6676,2416.3668,0.7362,0.2506,0.1949
4,1587.6026,6474512.9169,2544.5064,0.7242,0.2616,0.1882
5,1687.2565,8353327.4536,2890.2124,0.7134,0.2486,0.1950
6,1449.8389,4730666.8626,2175.0096,0.7384,0.2242,0.1761
7,1574.6586,5431581.6251,2330.5754,0.7035,0.2350,0.1886
8,1579.2080,6057127.9173,2461.1233,0.6798,0.2418,0.1801


In [35]:
# ensemble model
bagged_best = ensemble_model(best)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1300.2241,4296551.9894,2072.8126,0.7492,0.2110,0.1510
1,1465.3247,7166947.3942,2677.1155,0.6910,0.2163,0.1535
2,1319.6182,5114473.4830,2261.5202,0.7361,0.2244,0.1495
3,1341.9966,5168969.7712,2273.5368,0.7664,0.2161,0.1514
4,1350.9225,7088175.2479,2662.3627,0.6980,0.2348,0.1458
5,1396.2476,7785490.7474,2790.2492,0.7329,0.2124,0.1515
6,1191.4658,3646527.6332,1909.5883,0.7983,0.1957,0.1414
7,1291.9161,4455700.4996,2110.8530,0.7568,0.2053,0.1487
8,1361.3005,5238167.0324,2288.7042,0.7231,0.2169,0.1478


In [36]:
predict_model(bagged_best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,1329.9934,5299183.8871,2301.9956,0.7554,0.2142,0.1506


,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,...,Dep_Time_hour_23,Dep_Time_hour_3,Dep_Time_hour_4,Dep_Time_hour_5,Dep_Time_hour_6,Dep_Time_hour_7,Dep_Time_hour_8,Dep_Time_hour_9,Price,Label
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10577.0,10072.843214
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6206.0,7344.234543
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10919.0,10919.000000
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13759.0,13954.693110
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17234.0,15318.589000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3200,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6152.0,8268.903811
3201,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14781.0,14292.856419
3202,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4049.0,4417.449674
3203,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9166.0,12266.948751


In [37]:
predictions = predict_model(bagged_best, data=test)
predictions.head()

,Airline,Source,Destination,Route,Dep_Time,Additional_Info,Stops,Label
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,17:30,No info,1,14471.890510
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,06:20,No info,1,4242.744774
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,19:15,In-flight meal not included,1,10564.031132
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,08:00,No info,1,11188.584073
4,Air Asia,Banglore,Delhi,BLR → DEL,23:55,No info,0,4892.411104


In [24]:
save_model(best, 'my_best_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='Price',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy='...
                  RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                        criterion='mse', max_depth=None,
                                        max_features='auto', max_leaf_nodes=None,
                                        max_samples=None,
                             

In [25]:
loaded_model = load_model('my_best_pipeline')
print(loaded_model)

Transformation Pipeline and Model Successfully Loaded
Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[], target='Price',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='...
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mse', max_depth=None,
                                       max_features='auto', max_leaf_nodes=None,
                                       max_sampl

In [26]:
predictions.head()

,Airline,Source,Destination,Route,Dep_Time,Additional_Info,Stops,Label
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,17:30,No info,1,14745.712351
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,06:20,No info,1,4226.000000
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,19:15,In-flight meal not included,1,10637.871365
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,08:00,No info,1,11027.850045
4,Air Asia,Banglore,Delhi,BLR → DEL,23:55,No info,0,4885.938694


In [38]:
submission_01 = predictions['Label'].astype("int")

In [39]:
submission_0.to_excel("F:\Hackathon\MachineHack\Flight_Ticket_Participant_Datasets\submission_01.xlsx", index=False)